## ReAct Agent

ReAct is a pattern that combines Reasoning and Acting. It is a way to make an agent more flexible and adaptable. The reasoning part is the agent's ability to think about the problem and the acting part is the agent's ability to take action using the tools provided to it.

In the last session, we created a tool calling agent that could call a tool to calculate the area of a rectangle. In this session, we will create a ReAct agent will be able to utilize some of this tool calling logic to take actions.


### Getting Started

To get started, let's bring in some of the code from the las session in here so we can make use of it.

In [1]:
from typing import Callable, Dict, Any
import json

def extract_function_metadata(function: Callable) -> Dict[str, Any]:
    """
    Creates a metadata dictionary from a function's signature.
    
    Parameters:
        function: The target function to analyze
        
    Returns:
        A dictionary containing the function's name, documentation, and parameter specifications
    """
    # Initialize the basic structure
    metadata = {
        "name": function.__name__,
        "description": function.__doc__,
        "parameters": {"properties": {}}
    }
    
    # Extract parameter types (excluding return annotation)
    parameter_types = {
        param_name: {"type": param_type.__name__}
        for param_name, param_type in function.__annotations__.items()
        if param_name != "return"
    }
    
    # Add parameters to metadata
    metadata["parameters"]["properties"] = parameter_types
    return metadata

def convert_argument_types(tool_invocation: Dict[str, Any], function_spec: Dict[str, Any]) -> Dict[str, Any]:
    """
    Ensures all arguments match their expected types according to the function specification.
    
    Parameters:
        tool_invocation: Dictionary with the tool name and arguments
        function_spec: Dictionary containing the expected parameter types
        
    Returns:
        Updated tool invocation with correctly typed arguments
    """
    expected_params = function_spec["parameters"]["properties"]
    
    # Type conversion mapping
    type_converters = {
        "int": int,
        "str": str,
        "bool": bool,
        "float": float
    }
    
    # Convert each argument to its expected type if needed
    for arg_name, arg_value in tool_invocation["arguments"].items():
        target_type = expected_params[arg_name].get("type")
        if not isinstance(arg_value, type_converters[target_type]):
            tool_invocation["arguments"][arg_name] = type_converters[target_type](arg_value)
            
    return tool_invocation

class Tool:
    """
    Wrapper class that encapsulates a function as a callable tool.
    
    Attributes:
        name: Tool identifier
        function: The underlying function
        specification: JSON-formatted function metadata
    """
    def __init__(self, name: str, function: Callable, specification: str):
        self.name = name
        self.function = function
        self.specification = specification
        
    def __str__(self):
        return self.specification
        
    def execute(self, **kwargs):
        """
        Runs the wrapped function with the provided arguments.
        
        Parameters:
            **kwargs: Arguments to pass to the function
            
        Returns:
            The function's result
        """
        return self.function(**kwargs)

def tool(function: Callable) -> Tool:
    """
    Decorator that transforms a regular function into a Tool instance.
    
    Parameters:
        function: The function to convert into a tool
        
    Returns:
        A fully configured Tool object
    """
    def create_tool():
        metadata = extract_function_metadata(function)
        return Tool(
            name=metadata.get("name"),
            function=function,
            specification=json.dumps(metadata)
        )
    
    return create_tool()

In [2]:
import json
import re
from typing import List, Dict, Any, Callable
from colorama import init, Fore, Style

# Initialize colorama
init(autoreset=True)

class ToolCallingAgent:
    """
    An agent that integrates language models with function-calling tools.
    
    This class manages the interaction between a language model and a set of tools,
    allowing the model to decide when to call functions and processing the results.
    """
    
    def __init__(self, llm: Callable, system_prompt: str, tools: List[Any]):
        """
        Initialize an AI agent with a language model and tools.
        
        Args:
            llm: Function that takes a prompt string and returns a response
            system_prompt: Instructions for guiding the language model's behavior
            tools: List of Tool objects that the agent can use
        """
        self.model = llm
        self.system_prompt = system_prompt
        self.tools = {tool.name: tool for tool in tools}
        self.conversation_history = []
        print(f"{Fore.GREEN}ToolCallingAgent initialized with {len(tools)} tools{Style.RESET_ALL}")
        
    def format_tools_for_prompt(self) -> str:
        """Format all available tools into a format the language model can understand."""
        tools_json = []
        for tool in self.tools.values():
            try:
                # Use the specification provided by the @tool decorator
                tools_json.append(json.loads(tool.specification))
            except (json.JSONDecodeError, AttributeError):
                # Fallback for tools without proper specification
                tools_json.append({
                    "name": tool.name,
                    "description": getattr(tool, "description", "No description available"),
                    "parameters": {"properties": {}}
                })
        
        print(f"{Fore.CYAN}Formatted {len(tools_json)} tools for LLM prompt{Style.RESET_ALL}")
        return f"<tools>\n{json.dumps(tools_json, indent=2)}\n</tools>"
    
    def extract_tool_calls(self, response: str) -> List[Dict[str, Any]]:
        """
        Parse the language model's response to extract tool call requests.
        
        Args:
            response: The text response from the language model
            
        Returns:
            A list of tool call dictionaries with 'name' and 'arguments' keys
        """
        tool_calls = []
        pattern = r"<tool_call>(.*?)</tool_call>"
        matches = re.findall(pattern, response, re.DOTALL)
        
        for match in matches:
            try:
                tool_call = json.loads(match.strip())
                if "name" in tool_call and "arguments" in tool_call:
                    tool_calls.append(tool_call)
            except json.JSONDecodeError:
                continue
        
        if tool_calls:
            print(f"{Fore.YELLOW}Extracted {len(tool_calls)} tool call(s) from LLM response{Style.RESET_ALL}")
        else:
            print(f"{Fore.YELLOW}No tool calls found in LLM response{Style.RESET_ALL}")
                
        return tool_calls
    
    def execute_tool(self, tool_call: Dict[str, Any]) -> Any:
        """
        Execute a tool based on the model's request.
        
        Args:
            tool_call: Dictionary with 'name' and 'arguments' for the tool
            
        Returns:
            The result from executing the tool
        """
        tool_name = tool_call.get("name")
        arguments = tool_call.get("arguments", {})
        
        if tool_name not in self.tools:
            print(f"{Fore.RED}Error: Tool '{tool_name}' not found{Style.RESET_ALL}")
            return f"Error: Tool '{tool_name}' not found"
            
        tool = self.tools[tool_name]
        print(f"{Fore.MAGENTA}Executing tool: {tool_name} with arguments: {json.dumps(arguments)}{Style.RESET_ALL}")
        
        # Validate and convert argument types using the tool's specification
        try:
            if hasattr(tool, "specification"):
                tool_spec = json.loads(tool.specification)
                validated_args = self.convert_argument_types(
                    {"arguments": arguments}, 
                    tool_spec
                )["arguments"]
                arguments = validated_args
                print(f"{Fore.BLUE}Arguments validated and converted to appropriate types{Style.RESET_ALL}")
        except (json.JSONDecodeError, AttributeError, KeyError) as e:
            print(f"{Fore.RED}Error validating arguments: {str(e)}{Style.RESET_ALL}")
            # Continue with original arguments if validation fails
            pass
            
        try:
            # Handle execution based on the tool interface
            # First try the execute method for tools created with the @tool decorator
            if hasattr(tool, "execute"):
                print(f"{Fore.GREEN}Calling tool.execute() method{Style.RESET_ALL}")
                return tool.execute(**arguments)
            # Then try the function attribute which is used by the @tool decorator
            elif hasattr(tool, "function"):
                print(f"{Fore.GREEN}Calling tool.function() method{Style.RESET_ALL}")
                return tool.function(**arguments)
            # Fall back to run method
            elif hasattr(tool, "run"):
                print(f"{Fore.GREEN}Calling tool.run() method{Style.RESET_ALL}")
                return tool.run(**arguments)
            # Last resort: call the tool directly if it's callable
            elif callable(tool):
                print(f"{Fore.GREEN}Calling tool directly{Style.RESET_ALL}")
                return tool(**arguments)
            else:
                print(f"{Fore.RED}Error: Tool '{tool_name}' is not callable{Style.RESET_ALL}")
                return f"Error: Tool '{tool_name}' is not callable"
        except Exception as e:
            print(f"{Fore.RED}Error executing {tool_name}: {str(e)}{Style.RESET_ALL}")
            return f"Error executing {tool_name}: {str(e)}"
    
    def convert_argument_types(self, tool_call: Dict[str, Any], tool_spec: Dict[str, Any]) -> Dict[str, Any]:
        """
        Convert arguments to their expected types based on tool specification.
        
        Args:
            tool_call: Dictionary containing arguments to convert
            tool_spec: Tool specification with expected types
            
        Returns:
            Updated tool call with properly typed arguments
        """
        if "parameters" not in tool_spec or "properties" not in tool_spec["parameters"]:
            return tool_call
            
        properties = tool_spec["parameters"]["properties"]
        
        # Standard type converters
        type_mapping = {
            "int": int,
            "str": str,
            "bool": bool,
            "float": float,
            "integer": int,
            "string": str,
            "boolean": bool,
            "number": float
        }
        
        for arg_name, arg_value in tool_call["arguments"].items():
            if arg_name in properties and "type" in properties[arg_name]:
                expected_type = properties[arg_name]["type"]
                
                if expected_type in type_mapping:
                    converter = type_mapping[expected_type]
                    try:
                        # Only convert if types don't match
                        if not isinstance(arg_value, converter):
                            print(f"{Fore.BLUE}Converting argument '{arg_name}' from {type(arg_value).__name__} to {expected_type}{Style.RESET_ALL}")
                            tool_call["arguments"][arg_name] = converter(arg_value)
                    except (ValueError, TypeError) as e:
                        print(f"{Fore.RED}Type conversion error for '{arg_name}': {str(e)}{Style.RESET_ALL}")
                        # Keep original value if conversion fails
                        pass
                        
        return tool_call
    
    def run(self, user_input: str) -> str:
        print(f"{Fore.WHITE}{Style.BRIGHT}=== Starting agent run with user input: '{user_input}' ==={Style.RESET_ALL}")
        # Add user input to conversation history
        self.conversation_history.append({"role": "user", "content": user_input})

        # Build the messages list for OpenAI API
        messages = [
            {"role": "system", "content": self.system_prompt + "\n\n" + self.format_tools_for_prompt()}
        ]
        
        # Add conversation history
        for message in self.conversation_history:
            messages.append({"role": message["role"], "content": message["content"]})

        # Get response from language model
        print(f"{Fore.CYAN}Calling language model...{Style.RESET_ALL}")
        response = self.model(model="gpt-4o", messages=messages)
        model_response = response.choices[0].message.content
        print(f"{Fore.CYAN}Received response from language model ({len(model_response)} chars){Style.RESET_ALL}")

        # Extract tool calls from response
        tool_calls = self.extract_tool_calls(model_response)
        
        # If no tool calls, return the response directly
        if not tool_calls:
            print(f"{Fore.GREEN}No tool calls needed. Returning response.{Style.RESET_ALL}")
            final_response = model_response
        else:
            # Execute tools and collect results
            tool_results = []
            for i, tool_call in enumerate(tool_calls):
                print(f"{Fore.MAGENTA}{Style.BRIGHT}Executing tool call {i+1}/{len(tool_calls)}{Style.RESET_ALL}")
                result = self.execute_tool(tool_call)
                tool_results.append({
                    "tool": tool_call.get("name"),
                    "arguments": tool_call.get("arguments"),
                    "result": result
                })
            
            # Format tool results
            results_text = "Tool results:\n"
            for res in tool_results:
                result_str = str(res["result"])
                if isinstance(res["result"], (dict, list)):
                    try:
                        result_str = json.dumps(res["result"], indent=2)
                    except:
                        pass
                    
                results_text += f"- {res['tool']}{json.dumps(res['arguments'])}: {result_str}\n"
            
            print(f"{Fore.BLUE}Formatted tool results{Style.RESET_ALL}")
            
            # Create a new message with original response and tool results
            messages.append({"role": "assistant", "content": model_response})
            messages.append({"role": "user", "content": results_text})
            
            # Get final response from language model with tool results
            print(f"{Fore.CYAN}Calling language model with tool results...{Style.RESET_ALL}")
            final_response_obj = self.model(model="gpt-4o", messages=messages)
            final_response = final_response_obj.choices[0].message.content
            print(f"{Fore.CYAN}Received final response from language model ({len(final_response)} chars){Style.RESET_ALL}")
        
        # Add final response to conversation history
        self.conversation_history.append({
            "role": "assistant", 
            "content": final_response
        })
        
        print(f"{Fore.WHITE}{Style.BRIGHT}=== Agent run completed ===={Style.RESET_ALL}")
        return final_response
        
    def reset_conversation(self):
        """Clear the conversation history."""
        print(f"{Fore.GREEN}Conversation history reset{Style.RESET_ALL}")
        self.conversation_history = []

We can then bring in some other imports we will need for this session and create the LLM model we'll be using.

In [3]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

True

In [4]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### ReAct Agent System Prompt

In [23]:
REACT_SYSTEM_PROMPT = """
# ReAct Agent: Reasoning and Acting Framework

You are an AI assistant that follows the ReAct (Reasoning and Acting) framework to solve problems. Your thinking process is structured in a clear cycle: Thought → Action → Observation.

## How You Operate

1. **THOUGHT**: Carefully reason about the problem and determine if a tool call is necessary
2. **ACTION**: Execute tools based on your reasoning, following the proper format
3. **OBSERVATION**: Analyze the results to inform your final response

## Available Tools

You have access to the following functions to help users:

<tools>
__TOOLS__
</tools>

## Tool Calling Format

When you decide to use a tool, format your call exactly as follows:

<tool_call>
{"name": "function_name", "arguments": {"param1": "value1", "param2": "value2"}}
</tool_call>

For example, if you have a tool with this definition:

<tool_call>
{
    "name": "add_two_numbers",
    "description": "Used to add two numbers together",
    "parameters": {
        "properties": {
            "a": {
                "type": "int"
            },
            "b": {
                "type": "int"
            }
        }
    }
}
</tool_call>

Your tool call should look like:

<tool_call>
{"name": "add_two_numbers", "arguments": {"a": 1, "b": 2}}
</tool_call>

## Interaction Flow

Follow this structured approach to every user request:

1. <thought>Your detailed reasoning about how to approach the problem</thought>
2. <tool_call>Your properly formatted tool call if needed</tool_call>

After your tool call, you will receive:

<observation>Results from the tool execution</observation>

Then provide your final answer:

<response>Your complete, helpful answer based on the tool results</response>

## Example Interaction

<question>What's the sum of 10 and 20?</question>

<thought>To answer this question, I need to get the sum of 10 and 20. I should use a tool to perform this calculation.</thought>
<tool_call>{"name": "add_two_numbers", "arguments": {"a": 10, "b": 20}}</tool_call>

[System provides tool result]
<observation>{"result": 30}</observation>

<response>The sum of 10 and 20 is 30.</response>

## Important Guidelines

- Always begin with a <thought> tag to show your reasoning process
- Carefully inspect the function signature before making a tool call
- Pay special attention to parameter types and requirements
- Don't make assumptions about parameter values
- Use proper JSON syntax in your tool calls
- If a query doesn't require tools, respond directly:
  <thought>This question doesn't require any tool use because...</thought>
  <response>Your helpful answer here...</response>
- For complex tasks that require multiple tools, use one tool at a time
- If you receive unclear results, think about alternative approaches
"""

### Explanation of the ReAct Agent System Prompt

This system prompt creates an AI agent that follows the Reasoning and Acting (ReAct) framework. Here's a breakdown of how it works and how to customize it:

### XML Tags

The prompt uses several custom XML tags to structure the agent's thought process and actions:

1. `<thought>...</thought>` - Contains the agent's reasoning process before taking any actions
2. `<tool_call>...</tool_call>` - Wraps the JSON for calling a specific function/tool
3. `<observation>...</observation>` - Contains the results returned from tool execution
4. `<response>...</response>` - Contains the agent's final answer to the user
5. `<tools>...</tools>` - Defines all available tools the agent can use

### The `__TOOLS__` Placeholder

The `__TOOLS__` placeholder is where you'll inject the actual tool definitions before using the prompt. Here's how to replace it:

1. Generate JSON definitions for each tool you want to make available
2. Format each tool as a JSON object with `name`, `description`, and `parameters`
3. Replace `__TOOLS__` with the string representation of these tool definitions

For example, if you have two tools, you might replace it like this:

```python
tools = [
    {
        "name": "calculate_area",
        "description": "Calculate the area of a rectangle",
        "parameters": {
            "properties": {
                "length": {"type": "float"},
                "width": {"type": "float"}
            }
        }
    },
    {
        "name": "get_current_weather",
        "description": "Get the current weather for a location",
        "parameters": {
            "properties": {
                "location": {"type": "string"},
                "unit": {"type": "string"}
            }
        }
    }
]

# Convert tools to JSON string
tools_json = json.dumps(tools, indent=2)

# Replace __TOOLS__ in the prompt
final_prompt = REACT_SYSTEM_PROMPT.replace("__TOOLS__", tools_json)
```

### Agent Workflow

The prompt creates an agent that:

1. Takes in a user query
2. Thinks through the problem (in `<thought>` tags)
3. If needed, calls a tool (in `<tool_call>` tags)
4. Receives results (in `<observation>` tags)
5. Provides a final answer (in `<response>` tags)

This structured approach ensures transparency in the agent's reasoning process and makes debugging easier, as you can see exactly why the agent chose to call a specific tool and how it interpreted the results.

### Ceating Tools

We can now create some tools to use with our ReAct agent.



In [24]:
@tool
def add_two_numbers(a: int, b: int) -> int:
    """Used to add two numbers together"""
    return a + b

@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    """Used to calculate the area of a rectangle"""
    return length * width

We can view the specification of the tools we created by calling the `.specification` attribute on the tool.

In [25]:
add_two_numbers.specification

'{"name": "add_two_numbers", "description": "Used to add two numbers together", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}'

Now, let's get all the tool signatures or specifications and create a JSON string representation of them.

In [39]:
tools = [add_two_numbers, calculate_area_of_rectangle]
tools_mapping = {tool.name: tool for tool in tools}

tools_specifications = "".join([tool.specification for tool in tools])

In [27]:
print(tools_specifications)

{"name": "add_two_numbers", "description": "Used to add two numbers together", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}{"name": "calculate_area_of_rectangle", "description": "Used to calculate the area of a rectangle", "parameters": {"properties": {"length": {"type": "float"}, "width": {"type": "float"}}}}


We can now add this signature into our system prompt.

In [28]:
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT.replace("__TOOLS__", tools_specifications)

In [29]:
print(REACT_SYSTEM_PROMPT)


# ReAct Agent: Reasoning and Acting Framework

You are an AI assistant that follows the ReAct (Reasoning and Acting) framework to solve problems. Your thinking process is structured in a clear cycle: Thought → Action → Observation.

## How You Operate

1. **THOUGHT**: Carefully reason about the problem and determine if a tool call is necessary
2. **ACTION**: Execute tools based on your reasoning, following the proper format
3. **OBSERVATION**: Analyze the results to inform your final response

## Available Tools

You have access to the following functions to help users:

<tools>
{"name": "add_two_numbers", "description": "Used to add two numbers together", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}{"name": "calculate_area_of_rectangle", "description": "Used to calculate the area of a rectangle", "parameters": {"properties": {"length": {"type": "float"}, "width": {"type": "float"}}}}
</tools>

## Tool Calling Format

When you decide to use a tool, forma

### First Testing

In [30]:
user_query = "The sum of 10 and 20 is the width of a rectangle that is 100 units long. What is the area of the rectangle?"

user_prompt = [
    {"role": "system", "content": REACT_SYSTEM_PROMPT},
    {"role": "user", "content": f"<question>{user_query}</question>"}
]

In [34]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=user_prompt,
)

formatted_response = response.choices[0].message.content

In [35]:
print(formatted_response)

<thought>To find the area of the rectangle, I first need to determine the width, which is given as the sum of 10 and 20. I should first calculate this sum and then use the calculated width and the given length to find the area of the rectangle.</thought>
<tool_call>{"name": "add_two_numbers", "arguments": {"a": 10, "b": 20}}</tool_call>


Let's now execute the tool call that the LLM has suggested.


In [37]:
def transform_response_to_dict(response_content: str) -> dict:
    """
    Extracts all <tool_call> JSON blocks from the response_content and returns them as a list of dicts.
    Handles errors gracefully.
    """
    import re
    import json

    tool_calls = []
    try:
        # Find all <tool_call>...</tool_call> blocks
        matches = re.findall(r"<tool_call>\s*(\{.*?\})\s*</tool_call>", response_content, re.DOTALL)
        if not matches:
            raise ValueError("No <tool_call> JSON found in response.")

        for match in matches:
            try:
                tool_calls.append(json.loads(match))
            except json.JSONDecodeError as e:
                tool_calls.append({
                    "name": "JSONDecodeError",
                    "message": f"Error decoding JSON: {str(e)}",
                    "raw": match
                })
        return tool_calls
    except Exception as e:
        return [{
            "name": type(e).__name__,
            "message": str(e),
            "stack": None
        }]

In [38]:
transformed_response = transform_response_to_dict(formatted_response)
print(transformed_response)

[{'name': 'add_two_numbers', 'arguments': {'a': 10, 'b': 20}}]


In [46]:
tool_result = ""

for tool_call in transformed_response:
    tool_name = tool_call.get("name")
    tool = tools_mapping.get(tool_name)
    if tool:
        tool_result = tool.execute(**tool_call.get("arguments", {}))
        print(f"Tool {tool_name} executed with result: {tool_result}")
    else:
        print(f"Tool {tool_name} not found in tools_mapping")

Tool add_two_numbers executed with result: 30


We then send the tool results back to the LLM as part of the prompts

In [51]:
user_prompt.append({"role": "user", "content": f"<observation>{tool_result}</observation>"})

In [52]:
user_prompt

[{'role': 'system',
  'content': '\n# ReAct Agent: Reasoning and Acting Framework\n\nYou are an AI assistant that follows the ReAct (Reasoning and Acting) framework to solve problems. Your thinking process is structured in a clear cycle: Thought → Action → Observation.\n\n## How You Operate\n\n1. **THOUGHT**: Carefully reason about the problem and determine if a tool call is necessary\n2. **ACTION**: Execute tools based on your reasoning, following the proper format\n3. **OBSERVATION**: Analyze the results to inform your final response\n\n## Available Tools\n\nYou have access to the following functions to help users:\n\n<tools>\n{"name": "add_two_numbers", "description": "Used to add two numbers together", "parameters": {"properties": {"a": {"type": "int"}, "b": {"type": "int"}}}}{"name": "calculate_area_of_rectangle", "description": "Used to calculate the area of a rectangle", "parameters": {"properties": {"length": {"type": "float"}, "width": {"type": "float"}}}}\n</tools>\n\n## Tool

In [55]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=user_prompt,
)

From the output above, we can see the LLM now wants to calculate the area of the rectangle. This is since the tool call has been executed and the result has been returned for the addition of 10 and 20. Powered with this information, the LLM can now calculate the area of the rectangle.

This is the Observation bit of a ReAct agent. Action, Observation, Thought, Response if no further tool calls are needed.

In [56]:
formatted_response = response.choices[0].message.content
print(formatted_response)

<response>The sum of 10 and 20 is 30, which is the width of the rectangle. To find the area of the rectangle, I will multiply the width by the length. Let's calculate that.</response>

<thought>The width of the rectangle is 30 units and the length is 100 units. I should use the tool to calculate the area of the rectangle.</thought>
<tool_call>{"name": "calculate_area_of_rectangle", "arguments": {"length": 100, "width": 30}}</tool_call>


In [57]:
transformed_response = transform_response_to_dict(formatted_response)
print(transformed_response)

[{'name': 'calculate_area_of_rectangle', 'arguments': {'length': 100, 'width': 30}}]


In [58]:
tool_result = ""

for tool_call in transformed_response:
    tool_name = tool_call.get("name")
    tool = tools_mapping.get(tool_name)
    if tool:
        tool_result = tool.execute(**tool_call.get("arguments", {}))
        print(f"Tool {tool_name} executed with result: {tool_result}")
    else:
        print(f"Tool {tool_name} not found in tools_mapping")

Tool calculate_area_of_rectangle executed with result: 3000


In [59]:
user_prompt.append({"role": "user", "content": f"<observation>{tool_result}</observation>"})

In [60]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=user_prompt,
)

In [61]:
formatted_response = response.choices[0].message.content
print(formatted_response)

<response>The area of the rectangle with a length of 100 units and a width of 30 units is 3000 square units.</response>


In [67]:
import json
import re
from typing import Callable, List, Dict, Any
from colorama import init, Fore, Style

init(autoreset=True)

class Tool:
    def __init__(self, name: str, function: Callable, specification: str):
        self.name = name
        self.function = function
        self.specification = specification

    def execute(self, **kwargs):
        return self.function(**kwargs)

def extract_function_metadata(function: Callable) -> Dict[str, Any]:
    metadata = {
        "name": function.__name__,
        "description": function.__doc__,
        "parameters": {"properties": {}}
    }
    parameter_types = {
        param_name: {"type": param_type.__name__}
        for param_name, param_type in function.__annotations__.items()
        if param_name != "return"
    }
    metadata["parameters"]["properties"] = parameter_types
    return metadata

def tool(function: Callable) -> Tool:
    metadata = extract_function_metadata(function)
    return Tool(
        name=metadata.get("name"),
        function=function,
        specification=json.dumps(metadata)
    )

class ReActAgent:
    def __init__(self, llm: Callable, system_prompt: str, tools: List[Tool], max_iterations: int = 10):
        self.llm = llm
        self.system_prompt = system_prompt
        self.tools = {tool.name: tool for tool in tools}
        self.conversation_history = []
        self.max_iterations = max_iterations

    def format_tools_for_prompt(self) -> str:
        tools_json = [json.loads(tool.specification) for tool in self.tools.values()]
        return f"<tools>\n{json.dumps(tools_json, indent=2)}\n</tools>"

    def extract_tool_calls(self, response: str) -> List[Dict[str, Any]]:
        tool_calls = []
        pattern = r"<tool_call>(.*?)</tool_call>"
        matches = re.findall(pattern, response, re.DOTALL)
        for match in matches:
            try:
                tool_call = json.loads(match.strip())
                if "name" in tool_call and "arguments" in tool_call:
                    tool_calls.append(tool_call)
            except json.JSONDecodeError:
                print(f"{Fore.RED}Failed to decode tool call: {match}{Style.RESET_ALL}")
                continue
        return tool_calls

    def extract_final_response(self, response: str) -> str:
        match = re.search(r"<response>(.*?)</response>", response, re.DOTALL)
        if match:
            return match.group(1).strip()
        return None

    def convert_argument_types(self, tool_call: Dict[str, Any], tool_spec: Dict[str, Any]) -> Dict[str, Any]:
        if "parameters" not in tool_spec or "properties" not in tool_spec["parameters"]:
            return tool_call
        properties = tool_spec["parameters"]["properties"]
        type_mapping = {
            "int": int,
            "str": str,
            "bool": bool,
            "float": float,
            "integer": int,
            "string": str,
            "boolean": bool,
            "number": float
        }
        for arg_name, arg_value in tool_call["arguments"].items():
            if arg_name in properties and "type" in properties[arg_name]:
                expected_type = properties[arg_name]["type"]
                if expected_type in type_mapping:
                    converter = type_mapping[expected_type]
                    try:
                        if not isinstance(arg_value, converter):
                            print(f"{Fore.BLUE}Converting argument '{arg_name}' from {type(arg_value).__name__} to {expected_type}{Style.RESET_ALL}")
                            tool_call["arguments"][arg_name] = converter(arg_value)
                    except (ValueError, TypeError) as e:
                        print(f"{Fore.RED}Type conversion error for '{arg_name}': {str(e)}{Style.RESET_ALL}")
                        pass
        return tool_call

    def execute_tool(self, tool_call: Dict[str, Any]) -> Any:
        tool_name = tool_call.get("name")
        arguments = tool_call.get("arguments", {})
        if tool_name not in self.tools:
            print(f"{Fore.RED}Error: Tool '{tool_name}' not found{Style.RESET_ALL}")
            return f"Error: Tool '{tool_name}' not found"
        tool = self.tools[tool_name]
        print(f"{Fore.MAGENTA}Executing tool: {tool_name} with arguments: {json.dumps(arguments)}{Style.RESET_ALL}")
        try:
            tool_spec = json.loads(tool.specification)
            validated_args = self.convert_argument_types({"arguments": arguments}, tool_spec)["arguments"]
            arguments = validated_args
        except Exception as e:
            print(f"{Fore.RED}Error validating arguments: {str(e)}{Style.RESET_ALL}")
            pass
        try:
            result = tool.execute(**arguments)
            print(f"{Fore.GREEN}Tool '{tool_name}' executed successfully. Result: {result}{Style.RESET_ALL}")
            return result
        except Exception as e:
            print(f"{Fore.RED}Error executing {tool_name}: {str(e)}{Style.RESET_ALL}")
            return f"Error executing {tool_name}: {str(e)}"

    def run(self, user_input: str) -> str:
        self.conversation_history.append({"role": "user", "content": user_input})
        messages = [
            {"role": "system", "content": self.system_prompt + "\n\n" + self.format_tools_for_prompt()}
        ]
        for message in self.conversation_history:
            messages.append({"role": message["role"], "content": message["content"]})

        iterations = 0
        last_response = None

        while iterations < self.max_iterations:
            iterations += 1
            print(f"{Fore.CYAN}{Style.BRIGHT}--- Iteration {iterations} ---{Style.RESET_ALL}")

            # Get response from language model
            response = self.llm(model="gpt-4o", messages=messages)
            model_response = response.choices[0].message.content
            print(f"{Fore.YELLOW}Model response:\n{model_response}{Style.RESET_ALL}")

            # Check for final response
            final_response = self.extract_final_response(model_response)
            if final_response is not None:
                # print(f"{Fore.GREEN}{Style.BRIGHT}Final response found!{Style.RESET_ALL}")
                self.conversation_history.append({"role": "assistant", "content": model_response})
                return final_response

            # Extract tool calls from response
            tool_calls = self.extract_tool_calls(model_response)
            if not tool_calls:
                print(f"{Fore.GREEN}No tool calls found. Returning model response.{Style.RESET_ALL}")
                self.conversation_history.append({"role": "assistant", "content": model_response})
                return model_response

            # Execute tools and collect results
            tool_results = []
            for i, tool_call in enumerate(tool_calls):
                print(f"{Fore.MAGENTA}{Style.BRIGHT}Executing tool call {i+1}/{len(tool_calls)}{Style.RESET_ALL}")
                result = self.execute_tool(tool_call)
                tool_results.append({
                    "tool": tool_call.get("name"),
                    "arguments": tool_call.get("arguments"),
                    "result": result
                })

            # Format tool results for the next prompt
            results_text = "Tool results:\n"
            for res in tool_results:
                result_str = str(res["result"])
                if isinstance(res["result"], (dict, list)):
                    try:
                        result_str = json.dumps(res["result"], indent=2)
                    except Exception:
                        pass
                results_text += f"- {res['tool']}{json.dumps(res['arguments'])}: {result_str}\n"

            print(f"{Fore.BLUE}Tool results to be sent to LLM:\n{results_text}{Style.RESET_ALL}")

            # Add model response and tool results to messages for next iteration
            messages.append({"role": "assistant", "content": model_response})
            messages.append({"role": "user", "content": results_text})

        print(f"{Fore.RED}Max iterations reached without a final response.{Style.RESET_ALL}")
        return "Max iterations reached without a final response."

    def reset_conversation(self):
        print(f"{Fore.GREEN}Conversation history reset{Style.RESET_ALL}")
        self.conversation_history = []

In [68]:
from openai import OpenAI

# Define your tools
@tool
def add_two_numbers(a: int, b: int) -> int:
    """Used to add two numbers together"""
    return a + b

@tool
def calculate_area_of_rectangle(length: float, width: float) -> float:
    """Used to calculate the area of a rectangle"""
    return length * width

tools = [add_two_numbers, calculate_area_of_rectangle]

# Prepare your system prompt (replace __TOOLS__ as in your notebook)
tools_json = json.dumps([json.loads(t.specification) for t in tools], indent=2)
REACT_SYSTEM_PROMPT = REACT_SYSTEM_PROMPT.replace("__TOOLS__", tools_json)

# Create OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # Or use os.getenv

# Create the agent
agent = ReActAgent(
    llm=client.chat.completions.create,
    system_prompt=REACT_SYSTEM_PROMPT,
    tools=tools
)

# Run the agent
result = agent.run("The sum of 10 and 20 is the width of a rectangle that is 100 units long. What is the area of the rectangle?")
print(result)

--- Iteration 1 ---


Model response:
<thought>To find the area of the rectangle, I need to calculate its width by summing 10 and 20. Then, I will use the width and the given length to calculate the area of the rectangle using the appropriate tool.</thought>

<tool_call>{"name": "add_two_numbers", "arguments": {"a": 10, "b": 20}}</tool_call>
Executing tool call 1/1
Executing tool: add_two_numbers with arguments: {"a": 10, "b": 20}
Tool 'add_two_numbers' executed successfully. Result: 30
Tool results to be sent to LLM:
Tool results:
- add_two_numbers{"a": 10, "b": 20}: 30

--- Iteration 2 ---
Model response:
<observation>{"result": 30}</observation>

<thought>The width of the rectangle is 30 units. Now that I have both the length (100 units) and the width (30 units), I can calculate the area of the rectangle.</thought>
<tool_call>{"name": "calculate_area_of_rectangle", "arguments": {"length": 100.0, "width": 30.0}}</tool_call>
Executing tool call 1/1
Executing tool: calculate_area_of_rectangle with arguments